In [11]:
import os
import cv2
import numpy as np

In [12]:
root = './data/OCTID'

In [13]:
selected_catogeries = {
    'NO': 'Normal-Retinal-OCT-images', 
    'AMD': 'Age-related-Macular-Degeneration-Retinal-OCT-images',
    'DR': 'Diabetic-Retinopathy-Retinal-OCT-Images',
    'CSR': 'Central-serous-retinopathy-retinal-OCT-images',
    'MH': 'Macular-Hole-Retinal-OCT-images'
}


In [14]:
def removing_marks(image):
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    image_array = np.array(image)[:,:,0]
    mask1 = (image_array>100)
    # mask1 to image
    mask1 = mask1.astype(np.uint8)
    mask1 = cv2.cvtColor(mask1, cv2.COLOR_GRAY2BGR)

    kernel = np.ones((2,2), np.uint8)
    mask1 = cv2.dilate(mask1, kernel, iterations=1)

    mask[image.shape[0] - 150:image.shape[0] - 10, 10:150] = 255

    mask1 = np.array(mask1)[:,:,0]
    mask2 = mask*mask1

    restored_image = cv2.inpaint(image, mask2, 3, cv2.INPAINT_TELEA)
    return restored_image

In [ ]:
for label in selected_catogeries.keys():
    folder = selected_catogeries[label]
    print(folder, len(os.listdir(os.path.join(root,'original',folder))))
    for file in os.listdir(os.path.join(root,'original',folder)):
        image = cv2.imread(os.path.join(root,'original',folder,file))
        # resize to 500, 750, 3
        image = cv2.resize(image, (750, 500))
        image = removing_marks(image)
        # mkdir if not exist
        if not os.path.exists(os.path.join(root,'preprocessed',folder)):
            os.makedirs(os.path.join(root,'preprocessed',folder))
        cv2.imwrite(os.path.join(root,'preprocessed',folder,file), image)
        print(file, image.shape)
